In [167]:
import numpy as np
import cv2
import time
from pycuda import driver
from pycuda.compiler import SourceModule

TPB = 32
sigma=200
image = cv2.imread('test.bmp', cv2.IMREAD_GRAYSCALE)

In [168]:
def bilateralFilterCPU(image, sigma):
    result = np.zeros(image.shape)
    for x in range(1, image.shape[0]-1):
        for y in range(1, image.shape[1]-1):
            c = 0
            s = 0    
            for i in range(x-1, x+2):
                for j in range(y-1, y+2):

                    r = np.exp((-(image[i, j] - image[x,y]) ** 2) / sigma ** 2) 
                    
                    g = np.exp(-((i - x) ** 2 + (j - y) ** 2) / sigma ** 2)

                    c += g * r
                    s += g * r * image[i, j]

            result[x, y] = s / c

    for x in range(0, image.shape[0]):
        image[x,0]=image[x,1]
        image[x,image.shape[1]-1]=image[x,image.shape[1]-2]
        
    for y in range(0, image.shape[1]):
        image[0,y]=image[1,y]
        image[image.shape[0]-1,y]=image[image.shape[0]-2,y] 
    return image

In [175]:
start = time.time()
bilateral = cv2.bilateralFilter(image, 2, sigma, sigma)
cpu_t = time.time() - start
cv2.imwrite('res_cpu_cv2.bmp', bilateral)

start1 = time.time()
result = bilateralFilterCPU(image, sigma)
cpu_t1 = time.time() - start1
cv2.imwrite('res_cpu.bmp', result)

GPU = SourceModule(open("kernel.cu", "r").read())
kernel = GPU.get_function("kernel")
M,N = image.shape
grid = (int(np.ceil(M/TPB)), int(np.ceil(N/TPB)))

start2 = time.time()
tex = GPU.get_texref("tex")
driver.matrix_to_texref(image.astype(np.int32), tex, order="C")
result2 = np.zeros((M,N), dtype=np.int32)
kernel(driver.Out(result2), np.int32(M), np.int32(N), np.float32(sigma), texrefs=[tex], block=(TPB,TPB,1), grid=grid)
gpu_t = time.time() - start2
cv2.imwrite('res_gpu.bmp', result2.astype(np.uint8))

True

In [176]:
print("Время обработки изображения на GPU - ", gpu_t)
print("Время обработки изображения на CPU -", cpu_t1)
print("Время обработки изображения на CPU с помощью cv2.bilateralFilter -", cpu_t)

Время обработки изображения на GPU -  0.41199707984924316
Время обработки изображения на CPU - 176.5667769908905
Время обработки изображения на CPU с помощью cv2.bilateralFilter - 0.0039997100830078125
